# Basic Framework
First attempt to create a graph and a clustering of the words

In [1]:
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd
from tqdm import tqdm
# Math
import numpy as np
import scipy.stats
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa
import librosa.display
from scipy import sparse, stats, spatial
import scipy.sparse.linalg

from sklearn.decomposition import PCA

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd

# Cutting 

from cut_audio import *



%matplotlib inline

## 1. Import the Data
----
Use `N` random samples for each word 

In [2]:
N = 100

train_audio_path = '../Project/data/train/audio'
dirs = [f for f in os.listdir(train_audio_path) if isdir(join(train_audio_path, f))]
dirs.sort()

path = []
word = []
speaker = []
iteration = []

for direct in dirs:
    if not direct.startswith('_'):
        # Random selection of N files per folder 
        list_files = os.listdir(join(train_audio_path, direct))
        wave_selected  = list(np.random.choice([ f for f in list_files if f.endswith('.wav')],N,replace=False))
        
        # Extraction of file informations for dataframe
        word.extend(list(np.repeat(direct,N,axis=0)))
        speaker.extend([wave_selected[f].split('.')[0].split('_')[0] for f in range(N) ])
        iteration.extend([wave_selected[f].split('.')[0].split('_')[-1] for f in range(N) ])
        path.extend([train_audio_path + '/' + direct + '/' + wave_selected[f] for f in range(N)])


Initialize the dataframe of the original data

In [3]:
features_og = pd.DataFrame({('info','word',''): word,
                            ('info','speaker',''): speaker,
                            ('info','iteration',''): iteration,
                            ('info','path',''): path})
index_og = [('info','word',''),('info','speaker',''),('info','iteration','')]
#features_og.set_index(index_og,inplace=True)
print('Number of signals : ' + str(len(features_og)))
features_og.head()

Number of signals : 3000


info                                                                  
  iteration                                               path   speaker word
                                                                             
0         0  ../Project/data/train/audio/bed/627c0bec_nohas...  627c0bec  bed
1         0  ../Project/data/train/audio/bed/1df99a8a_nohas...  1df99a8a  bed
2         1  ../Project/data/train/audio/bed/2d92f18b_nohas...  2d92f18b  bed
3         0  ../Project/data/train/audio/bed/60472d26_nohas...  60472d26  bed
4         0  ../Project/data/train/audio/bed/c4cfbe43_nohas...  c4cfbe43  bed

## 2. Features Extraction
----
### 2.1 MFCC
A classical but reliable set a features

In [4]:
N_MFCC = 20

def compute_mfcc(filepath):
    ''' 
    Cuts the signal to reduce the silences and computes the MFCC
    '''  
    audio, sampling_rate = librosa.load(filepath, sr=None, mono=True)
    cut_version = cut_signal(audio)
    return librosa.feature.mfcc(y=cut_version,sr=sampling_rate)

In [5]:
stat_name= ['mean','std','skew','kurtosis','median']
col_names = [('mfcc',stat_name[i],j) for i in range(len(stat_name))  for j in range(N_MFCC)]
features_mfcc =pd.DataFrame(columns=pd.MultiIndex.from_tuples(col_names),index=features_og.index)
# sorting the columns in order to improve index performances (see lexsort errors)
features_mfcc.sort_index(axis=1,inplace=True,sort_remaining=True)

# MFCC FEATURES :
for w in tqdm(range(len(features_og)),total=len(features_og),unit='waves'):
    mfcc = compute_mfcc(features_og[('info','path')].iloc[w])
    features_mfcc.loc[w, ('mfcc', 'mean')] = np.mean(mfcc,axis=1)
    features_mfcc.loc[w, ('mfcc', 'std')] = np.std(mfcc,axis=1)
    features_mfcc.loc[w, ('mfcc', 'skew')] = scipy.stats.skew(mfcc,axis=1)
    features_mfcc.loc[w, ('mfcc', 'kurtosis')] = scipy.stats.kurtosis(mfcc,axis=1)
    features_mfcc.loc[w, ('mfcc', 'median')] = np.median(mfcc,axis=1)

features_og = features_og.merge(features_mfcc,left_index=True,right_index=True)
features_og.head()

100%|██████████| 3000/3000 [09:50<00:00,  5.08waves/s]


info                                                                    \
  iteration                                               path   speaker word   
                                                                                
0         0  ../Project/data/train/audio/bed/627c0bec_nohas...  627c0bec  bed   
1         0  ../Project/data/train/audio/bed/1df99a8a_nohas...  1df99a8a  bed   
2         1  ../Project/data/train/audio/bed/2d92f18b_nohas...  2d92f18b  bed   
3         0  ../Project/data/train/audio/bed/60472d26_nohas...  60472d26  bed   
4         0  ../Project/data/train/audio/bed/c4cfbe43_nohas...  c4cfbe43  bed   

       mfcc                                                   ...              \
   kurtosis                                                   ...         std   
          0        1        2         3         4         5   ...          10   
0  -1.05874 -1.32097 -1.11902  -1.25997 -0.849511 -0.929291   ...     11.6802   
1 -0.633586 -1.05611 -1.08586 -0.920467  -1.22205 -0.478814   ...      6.3033   
2  -1.41806 -0.75265   -1.246 -0.414955  -1.16298 -0.241393   ...     10.2024   
3  -1.46187 -1.09923 -1.36676 -0.606511  -0.54473  -0.48103   ...     3.21371   
4  -1.02178 -1.37959 -1.01953  -1.21673  -1.12551  -1.11922   ...     7.76927   

                                                                           \
                                                                            
        11       12       13       14       15       16       17       18   
0  16.4548  13.8252  4.87794  6.72855  6.10284  8.82942   4.1818  3.58219   
1   10.472  10.1622  3.38578  14.3874  4.94577  5.89547  6.26498   4.7197   
2   8.3409  4.85414  6.87863   9.6852  3.75003  3.11234  10.8917  2.89339   
3  6.09321  8.13679  2.63742  11.2553  6.98974  8.04308  4.34946  4.13938   
4  1.87411  7.10254  3.40943  5.80784  4.10858  5.62611  7.87066  5.03758   

            
            
        19  
0  10.9249  
1  10.3046  
2  3.28638  
3  5.96902  
4  3.11826  

[5 rows x 104 columns]

Saving the dataset features into a pickle to avoid to redo the computation on the training set

In [6]:
features_og.to_pickle('./Features Data/trainingFeatures.pickle')

## 3. Analysis

Importing the pickle containing the previously saved features

In [ ]:
features_og = pd.read_pickle('./Features Data/trainingFeatures.pickle')

In [ ]:
# drop features
features = pd.DataFrame(features_og)
features -= features.mean(axis=0)
features /= features.std(axis=0)

distances = spatial.distance.squareform(spatial.distance.pdist(features,'cosine'))

n=distances.shape[0]
kernel_width = distances.mean()
weights = np.exp(np.divide(-np.square(distances),kernel_width**2))
np.fill_diagonal(weights,0)

In [ ]:
fix, axes = plt.subplots(2, 2, figsize=(17, 8))
def plot(weights, axes):
    axes[0].spy(weights)
    axes[1].hist(weights[weights > 0].reshape(-1), bins=50);
plot(weights, axes[:, 0])

NEIGHBORS = 30

for i in range(weights.shape[0]):
    idx = weights[i,:].argsort()[:-NEIGHBORS]
    weights[i,idx] = 0
    weights[idx,i] = 0

plot(weights, axes[:, 1])
   

In [ ]:
degrees = np.sum(weights,axis=0)
laplacian = np.diag(degrees**-0.5) @ (np.diag(degrees) - weights) @ np.diag(degrees**-0.5)
laplacian = sparse.csr_matrix(laplacian)
plt.matshow(laplacian.todense());

In [ ]:
eigenvalues, eigenvectors = sparse.linalg.eigsh(A=laplacian,k=10,which='SM')

plt.plot(eigenvalues, '.-', markersize=15);

x = eigenvectors[:,1]
y = eigenvectors[:,2]

In [ ]:
fix, axes = plt.subplots(5, 5, figsize=(17, 8))
for i in range(1,6):
    for j in range(1,6):
        x = eigenvectors[:,i]
        y = eigenvectors[:,j]
        labels = np.sign(x)
        axes[i-1,j-1].scatter(x, y, c=labels, cmap='RdBu', alpha=0.5)